# String Algorithms Scratchpad

Exploring fuzzy matching and String Similarity tools to align datasets

#### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

/Users/sm/.pyenv/versions/3.10.0/lib/python3.10/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('max_seq_item', None)

In [3]:
from difflib import SequenceMatcher
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from thefuzz import fuzz
from thefuzz import process
import textdistance
import jaro
import jellyfish

/Users/sm/.pyenv/versions/3.10.0/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/Users/sm/.pyenv/versions/3.10.0/lib/python3.10/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [60]:
#natural language processing imports
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import treebank
import string

In [62]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('treebank')

[nltk_data] Downloading package punkt to /Users/sm/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/sm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/sm/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/sm/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/sm/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to /Users/sm/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

-----

##### Understanding FuzzyWuzzy + The Fuzz

Source: 
https://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/

https://github.com/seatgeek/thefuzz

https://itnext.io/string-similarity-the-basic-know-your-algorithms-guide-3de3d7346227

Simple Ratios

In [4]:
m = SequenceMatcher(None, "NEW YORK METS", "NEW YORK MEATS")
m.ratio()

0.9629629629629629

In [5]:
fuzz.ratio("NEW YORK METS", "NEW YORK MEATS")

96

In [6]:
fuzz.ratio("YANKEES", "NEW YORK YANKEES")

61

In [7]:
fuzz.ratio("NEW YORK METS", "NEW YORK YANKEES")

76

In [8]:
fuzz.ratio("YANKEES", "NEW YOR")

14

In [9]:
fuzz.ratio("YANKEES", "EW YORK")

29

In [10]:
fuzz.ratio("YANKEES", "YANKEES")

100

In [11]:
fuzz.ratio("this is a test", "this is a test!")

97

Partial Ratio

In [12]:
# A perfect Partial Match
fuzz.partial_ratio("YANKEES", "NEW YORK YANKEES")

100

In [13]:
fuzz.partial_ratio("NEW YORK METS", "NEW YORK YANKEES")

69

Token Sort Ratio

In [14]:
#without token sort
fuzz.ratio("New York Mets vs Atlanta Braves", "Atlanta Braves vs New York Mets")

45

In [15]:
#with token sort
fuzz.token_sort_ratio("New York Mets vs Atlanta Braves", "Atlanta Braves vs New York Mets")

100

In [16]:
fuzz.ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear")

91

In [17]:
fuzz.token_sort_ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear")

100

Token Set

In [18]:
s1 = "mariners vs angels"
s2 = "los angeles angels of anaheim at seattle mariners"

In [19]:
t0 = "angels mariners"
t1 = "angels mariners vs"
t2 = "angels mariners anaheim angeles at los of seattle"

In [20]:
# t0 = [SORTED_INTERSECTION]
# t1 = [SORTED_INTERSECTION] + [SORTED_REST_OF_STRING1]
# t2 = [SORTED_INTERSECTION] + [SORTED_REST_OF_STRING2]

In [21]:
fuzz.ratio(t0, t1)

91

In [22]:
fuzz.ratio(t0, t2)

47

In [23]:
fuzz.ratio(t1, t2)

51

In [24]:
fuzz.token_set_ratio("mariners vs angels", "los angeles angels of anaheim at seattle mariners")

91

In [25]:
fuzz.token_set_ratio("Sirhan, Sirhan", "Sirhan")

100

In [26]:
fuzz.token_sort_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")

84

In [27]:
fuzz.token_set_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")

100

Process?

In [28]:
choices = ["Atlanta Falcons", "New York Jets", "New York Giants", "Dallas Cowboys"]

In [29]:
process.extract("new york jets", choices, limit=2)

[('New York Jets', 100), ('New York Giants', 79)]

In [30]:
process.extract("new york jets", choices, limit=3)

[('New York Jets', 100), ('New York Giants', 79), ('Atlanta Falcons', 29)]

In [31]:
process.extractOne("cowboys", choices)

('Dallas Cowboys', 90)

In [32]:
#parameters to extractOne to make it use a specific scorer
# >>> process.extractOne("System of a down - Hypnotize - Heroin", songs)
#     ('/music/library/good/System of a Down/2005 - Hypnotize/01 - Attack.mp3', 86)
# >>> process.extractOne("System of a down - Hypnotize - Heroin", songs, scorer=fuzz.token_sort_ratio)
#     ("/music/library/good/System of a Down/2005 - Hypnotize/10 - She's Like Heroin.mp3", 61)

----

##### Types of Algorithms

Source: https://itnext.io/string-similarity-the-basic-know-your-algorithms-guide-3de3d7346227

Based on the properties of operations, string similarity algorithms can be classified into a bunch of domains. Let’s discuss a few of them,
Edit distance based: Algorithms falling under this category try to compute the number of operations needed to transforms one string to another. More the number of operations, less is the similarity between the two strings. One point to note, in this case, every index character of the string is given equal importance.

Token-based: In this category, the expected input is a set of tokens, rather than complete strings. The idea is to find the similar tokens in both sets. More the number of common tokens, more is the similarity between the sets. A string can be transformed into sets by splitting using a delimiter. This way, we can transform a sentence into tokens of words or n-grams characters. Note, here tokens of different length have equal importance.

Sequence-based: Here, the similarity is a factor of common sub-strings between the two strings. The algorithms, try to find the longest sequence which is present in both strings, the more of these sequences found, higher is the similarity score. Note, here combination of characters of same length have equal importance.

Hamming Distance (finding the places where the strings vary)

In [33]:
#the edit distance is 1 for only the difference being one letter different
textdistance.hamming('text', 'test')

1

In [34]:
#75% similar between text and test
textdistance.hamming.normalized_similarity('text', 'test')

0.75

In [35]:
textdistance.hamming('arrow', 'arow')

3

In [36]:
textdistance.hamming.normalized_similarity('arrow', 'arow')

0.4

Levenshtein Distance (finding the number of edits which will transform one string to another)

In [37]:
#number of edits it will take to transform one to the other
textdistance.levenshtein('arrow', 'arow')

1

In [38]:
textdistance.levenshtein.normalized_similarity('arrow', 'arow')

0.8

Jaro-Winkler 

"(1) they contain same characters, but within a certain distance from one another, and (2) the order of the matching characters is same."

In [39]:
# textdistance.jaro_winkler("mes", "messi")

In [40]:
# textdistance.jaro_winkler("crate", "crat")

In [41]:
# textdistance.jaro_winkler("crate", "atcr")

Jaccard Index (find the number of common tokens and divide it by the total number of unique tokens)

"We first tokenize the string by default space delimiter, to make words in the strings as tokens. Then we compute the similarity score." 

In [42]:
tokens_1 = "hello world".split()
tokens_2 = "world hello".split()

In [43]:
textdistance.jaccard(tokens_1 , tokens_2)

1.0

In [44]:
tokens_1 = "hello new world".split()
tokens_2 = "hello world".split()

In [45]:
textdistance.jaccard(tokens_1 , tokens_2)

0.6666666666666666

Sorensen-Dice 

"Falling under set similarity, the logic is to find the common tokens, and divide it by the total number of tokens present by combining both sets." 

In [46]:
tokens_1 = "hello world".split()
tokens_2 = "world hello".split()

In [47]:
textdistance.sorensen(tokens_1 , tokens_2)

1.0

In [48]:
tokens_1 = "hello new world".split()
tokens_2 = "hello world".split()

In [49]:
textdistance.sorensen(tokens_1 , tokens_2)

0.8

Ratcliff-Obershelp similarity

In [50]:
string1, string2 = "i am going home", "gone home"

In [51]:
textdistance.ratcliff_obershelp(string1, string2)

0.6666666666666666

In [52]:
string1, string2 = "helloworld", "worldhello"

In [53]:
textdistance.ratcliff_obershelp(string1, string2)

0.5

In [54]:
string1, string2 = "test", "text"

In [55]:
textdistance.ratcliff_obershelp(string1, string2)

0.75

In [56]:
string1, string2 = "mes", "simes"

In [57]:
textdistance.ratcliff_obershelp(string1, string2)

0.75

In [58]:
string1, string2 = "arrow", "arow"

In [59]:
textdistance.ratcliff_obershelp(string1, string2)

0.8888888888888888

-----

##### Natural Language Processing Exploration

In [63]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [64]:
stops = set(stopwords.words('english'))
print(stops)

{"didn't", "should've", 't', 'who', 'each', 'yours', "shouldn't", 'yourselves', 'is', 'nor', 'their', 'had', 'yourself', 'll', 'what', "won't", "hasn't", 'more', "haven't", 'my', 'not', "aren't", 'do', 'themselves', 're', 'were', 'under', 'for', 'needn', 'between', 'theirs', 'only', 'now', 'below', 'don', "isn't", "shan't", "weren't", 'being', 'ain', 'his', 'ourselves', "don't", 'has', 'in', 'wasn', 'all', 'hers', 'your', 'couldn', 'been', "you've", 'how', 'aren', 'against', 'are', 've', 'here', 'same', 'she', 'very', 'haven', 'her', 'but', 'once', 'those', 'didn', 'out', 'am', 'he', 'shan', 'a', 'that', 'about', 'them', 'after', 'into', 'until', 'some', 'there', 'wouldn', 'both', 'ours', 'doing', "you'd", 'at', "you're", "she's", 'have', 'such', 'which', 'as', 'on', 'hasn', 'did', 'of', 'our', 'then', "mustn't", 'mustn', 'd', 'm', 'just', 'herself', 'an', 'why', 's', 'i', "wasn't", 'no', 'so', 'through', 'they', 'because', 'weren', 'if', "couldn't", 'than', "hadn't", 'and', 'will', 'w

In [65]:
from nltk.tokenize import sent_tokenize, word_tokenize

data = "All work and no play makes jack a dull boy, all work and no play"
print(word_tokenize(data))

['All', 'work', 'and', 'no', 'play', 'makes', 'jack', 'a', 'dull', 'boy', ',', 'all', 'work', 'and', 'no', 'play']


In [66]:
sentence = """At eight o'clock on Thursday morning Arthur didn't feel very good."""
tokens = nltk.word_tokenize(sentence)
print(tokens)

['At', 'eight', "o'clock", 'on', 'Thursday', 'morning', 'Arthur', 'did', "n't", 'feel', 'very', 'good', '.']


In [67]:
tagged = nltk.pos_tag(tokens)
tagged[0:6]

[('At', 'IN'),
 ('eight', 'CD'),
 ("o'clock", 'NN'),
 ('on', 'IN'),
 ('Thursday', 'NNP'),
 ('morning', 'NN')]

In [68]:
entities = nltk.chunk.ne_chunk(tagged)
print(entities)

(S
  At/IN
  eight/CD
  o'clock/NN
  on/IN
  Thursday/NNP
  morning/NN
  (PERSON Arthur/NNP)
  did/VBD
  n't/RB
  feel/VB
  very/RB
  good/JJ
  ./.)


In [69]:
## tree map
# t = treebank.parsed_sents('wsj_0001.mrg')[0]
# t.draw()